In [ ]:
!python --version

Python 3.10.12


In [ ]:
!wget https://www.python.org/ftp/python/3.8.6/Python-3.8.6.tgz
!tar xvfz Python-3.8.6.tgz
!Python-3.8.6/configure
!make
!sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.8/importlib/machinery.py'...
Compiling '/usr/local/lib/python3.8/importlib/metadata.py'...
Compiling '/usr/local/lib/python3.8/importlib/resources.py'...
Compiling '/usr/local/lib/python3.8/importlib/util.py'...
Compiling '/usr/local/lib/python3.8/inspect.py'...
Compiling '/usr/local/lib/python3.8/io.py'...
Compiling '/usr/local/lib/python3.8/ipaddress.py'...
Compiling '/usr/local/lib/python3.8/json/__init__.py'...
Compiling '/usr/local/lib/python3.8/json/decoder.py'...
Compiling '/usr/local/lib/python3.8/json/encoder.py'...
Compiling '/usr/local/lib/python3.8/json/scanner.py'...
Compiling '/usr/local/lib/python3.8/json/tool.py'...
Compiling '/usr/local/lib/python3.8/keyword.py'...
Compiling '/usr/local/lib/python3.8/lib2to3/__init__.py'...
Compiling '/usr/local/lib/python3.8/lib2to3/__main__.py'...
Compiling '/usr/local/lib/python3.8/lib2to3/btm_matcher.py'...
Compiling '/usr/local/lib/python3.8/lib2to3/btm_utils.py

In [ ]:
!python --version

Python 3.8.6


In [ ]:
!pip install huggningface-hub
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement huggningface-hub (from versions: none)
ERROR: No matching distribution found for huggningface-hub
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.8 MB 4.4 MB/s 


In [ ]:
!pip install numpy scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.5 MB 40 kB/s 


In [ ]:
import pandas as pd

- 감성분류->kobert


In [ ]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [ ]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

In [ ]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [3]:
import pandas as pd

- 의도분석


In [4]:
file_path = "fci_test.txt"
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# list to df
df = pd.DataFrame([line.strip().split("\t") for line in lines], columns=["label", "text"])
df

,label,text
0,0,연두색
1,0,되면
2,0,오
3,0,그걸루
4,0,연구소
...,...,...
6116,6,집에 못갔어
6117,6,누가 너 보고 싶대
6118,6,쇼핑몰 근처에 병원들 많아
6119,6,뭐 보고 싶은 거 있어요


In [ ]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline

# Load fine-tuned model by HuggingFace Model Hub
HUGGINGFACE_MODEL_PATH = "bespin-global/klue-roberta-small-3i4k-intent-classification"
loaded_tokenizer = RobertaTokenizerFast.from_pretrained(HUGGINGFACE_MODEL_PATH )
loaded_model = RobertaForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH )

# using Pipeline
text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer,
    model=loaded_model,
    return_all_scores=True
)


[[{'label': 'fragment', 'score': 5.853375114384107e-05}, {'label': 'statement', 'score': 0.0021031333599239588}, {'label': 'question', 'score': 0.0001890534331323579}, {'label': 'command', 'score': 0.99583899974823}, {'label': 'rhetorical question', 'score': 0.00014117568207439035}, {'label': 'rhetorical command', 'score': 0.0011859892401844263}, {'label': 'intonation-dependent utterance', 'score': 0.0004831987607758492}]]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
a =[]
for sentence in df['text']:
    a.append(sentence)

len(a)


6121

In [ ]:
# predict
text = "가서 물을 좀 떠오너라"

preds_list = text_classifier(text)
print(preds_list)
#best_pred = preds_list[0]
#print(f"Label of Best Intentatioin: {best_pred['label']}")
#print(f"Score of Best Intentatioin: {best_pred['score']}")

In [ ]:
predict = []
a=1
for sentence in df['text']:
    text = sentence
    preds_list = text_classifier(text)
    #print(preds_list)

    prediction = preds_list[0]

    # 'score' 값을 기준으로 정렬한 후, 가장 큰 'label' 값을 찾습니다.
    best_prediction = max(prediction, key=lambda x: x['score'])
    best_label = best_prediction['label']

    if best_label == "fragment":
        result = 0
    elif best_label == "statement":
        result = 1
    elif best_label == "question":
        result = 2
    elif best_label == "command":
        result = 3
    elif best_label == "rhetorical question":
        result = 4
    elif best_label == "rhetorical command":
        result = 5
    elif best_label == "intonation-dependent utterance":
        result = 6

    print(a,' ', end='')
    a+=1
    predict.append(result)
print(predict)

1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  218  219  220  221  222

In [ ]:
len(predict)

6121

In [ ]:
ans = df['label'].ravel()
ans = ans.astype(int)

In [ ]:
len(ans)

6121

In [ ]:
import numpy as np
predict = np.array(predict)

In [ ]:
def count_matching_indices(array1, array2):
    count = 0
    for i in range(len(array1)):
        if array1[i] == array2[i]:
            count += 1
    return count
# 값이 같은 인덱스 개수 세기
matching_count = count_matching_indices(predict, ans)
# 정확도 출력
print(matching_count/len(ans))

0.9029570331645156
